# init

In [1]:
from pymongo import MongoClient

client = MongoClient()
db = client.izidb

In [2]:
from scripts import izi
reload(izi)

n_topics = 100

--------------------> english word frequencies loaded
--------------------> english word frequencies loaded


# get files list

In [3]:
root = u"../izi_data/"
filelist = izi.getFileList(root)
print len(filelist)
example =  filelist[0]
print example[len(root):]

195
1412 Standard English Review-en-ru-T.mxliff


In [4]:
def safe_str(value):
    if type(value) == str:
    # Ignore errors even if the string is not proper UTF-8 or has
    # broken marker bytes.
    # Python built-in function unicode() can do this.
        value = unicode(value, "utf-8", errors="ignore")
    else:
    # Assume the value object has proper __unicode__() method
        value = unicode(value)
    return value

# define functions

In [5]:
def insertDoc(path, root = root, prefix = ''):
    document = dict()
    document['title'] = prefix + '|' + path[len(root):].replace( ' ', '_')
    document['type'] = prefix
#     try:
    full_text = izi.loadText(path)
    full_text = safe_str(full_text)
    if len(full_text) > 200:
        tokens = izi.tokenize( full_text)
        document['full_text'] = full_text
        document['tokens'] = tokens
        topics =  izi.topicsFromTokens(izi.tokenize(full_text))
        semantic_vec = [0.] * n_topics
        for i in topics:
            semantic_vec[i[0]] = i[1]
        document['semantic_vec'] = semantic_vec
        current_id = db.documents.save(document)
        # create links
        cursor = db.documents.find()
        for doc in cursor:
            y = doc['semantic_vec']
            y_id = doc["_id"]
            if y_id != current_id:
                s = izi.getSimilarity( semantic_vec, y)
                db.similarities.insert({'source': current_id , 'target': y_id, 'value': s})
#     except:
#         print "#### ERROR: " + document['title']

# Brown

In [ ]:
root2 = u"../brown/"
import nltk
brown_ids = nltk.corpus.brown.fileids()
for i in brown_ids:
    f = open( root2 + i + ".txt", "w")
    f.write(nltk.corpus.brown.raw(i) )

In [ ]:
for f in izi.getFileList(root2):
    insertDoc(f, root = root2)

In [ ]:
# cursor = db.documents.find_one({ "_id": id})
# for document in cursor:
#     print document

# scripts

In [ ]:
import urllib
root_scripts = u"../scripts_movies/"


In [ ]:
f = open('raw_script_urls.txt', 'r')
for l in f.readlines():
    ll =  [ u.rstrip() for u in l.split(' +++$+++ ') ]
    title = ll[1]
    title = title.replace(" ", "_")
    link = ll[2]
    try:
        p = urllib.urlopen(link).read()
        if len(p) > 1000:
            print link + ' ' + title
            path = root_scripts + title + ".txt"
            file = open( path, "w")
            
            file.write( p )
            insertDoc(path, root = root_scripts)
        else:
            print 'TOO SHORT: ' + link + ' ' + title
    except:
        print "ERROR: " + title

In [ ]:
print izi.getFileList(root_scripts)

In [ ]:
fails = []
print len(izi.getFileList(root_scripts))
i = 0
for f in izi.getFileList(root_scripts):
    try:
        insertDoc(f, root = root_scripts)
        print i,
    except:
        fails.append(f)
        print "#",
    i += 1

# gutemberg

In [ ]:
import urllib
root_gut = u"../gutemberg/"
for f in izi.getFileList(root_gut):
    if f[-3:] == 'txt':
        print f
        insertDoc(f, root = root_gut)

# masc

In [10]:
root_masc = u"../masc/"
for f in izi.getFileList(root_masc):
    sub_root = f+'/'
    for ff in izi.getFileList(sub_root):
        print ff
        insertDoc(ff, root = sub_root, prefix = f[len(root_masc):])

../masc/fiction/Alices Adventures in Wonderland.txt
../masc/fiction/A_Wasted_Day.txt

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:17: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:25: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.



../masc/fiction/captured_moments.txt
../masc/fiction/hotel-california.txt
../masc/fiction/lw1.txt
../masc/fiction/Nathans_Bylichka.txt
../masc/fiction/The Adventures of Sherlock Holmes.txt
../masc/fiction/The Iliad of Homer.txt
../masc/fiction/The_Black_Willow.txt
../masc/law/A Book About Lawyers.txt
../masc/law/An Account of the Proceedings on the Trial.txt
../masc/law/Briefless Ballads and Legal Lyrics.txt
../masc/law/Business Hints for Men and Women.txt
../masc/law/Childrens Internet Protection Act.txt
../masc/law/Commentaries on the Laws of Englan.txt
../masc/law/John Marshall and the Constitution.txt
../masc/law/Law and Laughter.txt
../masc/law/Our Legal Heritag.txt
../masc/law/Popular Law-making.txt
../masc/law/Report of the Decision of the Supreme Court.txt
../masc/law/The Acts Of The General Assemblies of the.txt
../masc/law/The Acts of Uniformity.txt
../masc/law/The Constitution of the United States.txt
../masc/law/The Declaration of Independenc.txt
../masc/law/The Divine Rig

# add all documents

In [ ]:
fails = []
i = 0
maxx = 500
for f in filelist:
    if i < maxx:
        try:
            insertDoc(f)
            print i,
        except:
            fails.append(f)
            print "#",

        i += 1
    
print
print
print "%s fails" %len(fails)

In [ ]:
for f in fails:
    print f

# tests

In [ ]:
cursor = db.documents.find_one()
for document in cursor:
    print document
    
    
x = cursor["_id"]

db.documents.find_one({ "_id": x})["full_text"]

In [ ]:
db.similarities.find_one()

# reset collections

In [9]:
db.restaurants.drop()
db.documents.drop()
db.similarities.drop()